# Answering Business Questions
---

The aim of this project is to answer business questions coming from modified version of a Chinook database, containing information about a fictional digital music shop. In the database we can find artists of the songs, albums as well as data related to shop's employees and customers.

The business questions we want to answer are:
- what is the most popular genre among our customers in the USA?
- is there a significant sales performance difference among employees?
- what are the countries with the highest sales?
- how many customers does the company have in particular countries?
- what is the average value of sales per customer in each country?

The schema diagram for our dataset can be found under following [link](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg).

## Exploring the Data

We will start preparing our sql dataset to work. Then we will write a query to return information on the tables in the database.

The aim of the following project is to answer business questions using SQL skills.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type="table";

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We can see above all the tables in our dataset. Let's explore some tables in more detail.

In [3]:
%%sql
SELECT * FROM customer LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


The `customer` table contains company customers' information. We can find here where the customer is coming from, their phone numbers etc.

In [4]:
%%sql
SELECT * FROM invoice LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In `invoice` column we can find information regarding transaction details (customer identification number, invoice data etc.

In [5]:
%%sql
SELECT * FROM track LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In the `track` column we can see information regarding track name, composer, bytes etc.

## Most Popular Genre

Because the Chinook record store has signed a deal with new record label, one of our goals is to find which genres sell the most tracks in the USA. There is a `genre` table in our dataset, let's investigate it.

In [6]:
%%sql
SELECT * FROM genre;

 * sqlite:///chinook.db
Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


There are 25 various types of genres. We will need data from other tables in order to calculate the number of tracks sold in the USA. The `track` table is connected with `genre` table (the primary key is `genre_id`) and with `invoice_line` table (primary key: `track_id`). However, we will also need information related to country of customer's residence in order to indicate the number of tracks sold in the USA. 

In [7]:
%%sql

WITH tracks_sold_usa AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    COUNT(tsu.invoice_line_id) tracks_sold,
    ROUND(CAST(COUNT(tsu.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) FROM tracks_sold_usa), 2
    ) proportion_sold
FROM tracks_sold_usa tsu
INNER JOIN track t on t.track_id = tsu.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,proportion_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


The most popular genre among sold tracks in USA is Rock, for which over a half of tracks has been sold (0,53). Alternative & Punk and Metal are next, however they account for 0,12, which is quite a big gap from the first genre. Other genre do not indicate significant proportions in sales on the terriitory of the USA.

## Employee Sales Performance

In this part of the project we will investigate employee sales performance. Chinook's store customers are assigned to a sales support agent within the company while making first purchases. We would like to find out if any employee is performing either better or worse than others.

First, we will explore the `employee` column. The `title` column from `employee' table indicates the occupation within the company. Finally,  we will write a query which will find the total dollar amount of sales assigned to each support agent.

In [8]:
%%sql
SELECT * FROM employee LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [31]:
%%sql

WITH reported_sales AS
    (
    SELECT
        i.customer_id,
        c.support_rep_id,
        ROUND(SUM(i.total),2) total
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 1,2
    )
    
SELECT
    e.first_name || " " || e.last_name sales_agent,
    e.hire_date,
    ROUND(SUM(rs.total),2) total_sales
FROM reported_sales rs
INNER JOIN employee e ON e.employee_id = rs.support_rep_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


sales_agent,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


The total dollar amount of sales is the highest in case of Jane Peacock, however, she is employed longer than the others. The difference in sales is not high among sales agents. It seems to be that the hiring time plays the main role in differences.

## Sales by Country

Earlier in the project we found out the most popular genre in the USA. We would like to know what are the sales in the USA and other countries as well. In particular we are interested in each country's:
- total number of customers,
- total value of sales,
- average value of sales per customer.

The countries with only one customer will be grouped in one column called `Other`.

In [10]:
%%sql
SELECT MAX(DISTINCT(i.customer_id)) FROM invoice i

 * sqlite:///chinook.db
Done.


MAX(DISTINCT(i.customer_id))
59


In [35]:
%%sql

SELECT 
    COUNT(DISTINCT(i.customer_id)) total_customers, 
    c.country, 
    ROUND(SUM(i.total),2) total_sales,
    ROUND(ROUND(SUM(i.total),2) / COUNT(DISTINCT(i.customer_id)), 2) avg_sale_customer
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
GROUP BY 2
ORDER BY 1 DESC

 * sqlite:///chinook.db
Done.


total_customers,country,total_sales,avg_sale_customer
13,USA,1040.49,80.04
8,Canada,535.59,66.95
5,Brazil,427.68,85.54
5,France,389.07,77.81
4,Germany,334.62,83.66
3,United Kingdom,245.52,81.84
2,Czech Republic,273.24,136.62
2,India,183.15,91.58
2,Portugal,185.13,92.57
1,Argentina,39.6,39.6


The comapny has most customers and highest total sales in the USA. Other important customers are coming from markets in Canada, Brazil and France. Although there are only two customers in Czech Republic and total sales are not the highest, average sale per customer is the most significant. 

# Conclusions
---

To conclude, we analyzed data, using SQL queries, coming from a fictional digital music shop. We found out that the most popular genre among customers in the USA is rock. We investigated sale performance differences among employees and indicated that the workers are doing the job on similar performence level. Furthermore, we anayzed the sales for every country the company has customers in. Unsurprisingly, the company has the most customers in the USA, second was Canada, then Brazil and France. However, the average sales per customers varied among the countries, and happenned to be the highest in case of Czech Republic.